<a href="https://colab.research.google.com/github/VadimGrishin/Crowl_parse/blob/master/Copy_of_Parsing_less2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import requests
import pprint

**Task 1**

Доработать приложение по поиску авиабилетов, чтобы оно возвращало билеты по названию города, а не по IATA коду. Пункт отправления и пункт назначения должны передаваться в качестве параметров. Сделать форматированный вывод, который содержит в себе пункт отправления, пункт назначения, дату вылета, цену билета (можно добавить еще другие параметры по желанию)

In [0]:
flight_params = {
    'origin': 'Москва',
    'destination': 'Копенгаген'
}

In [5]:
# параметры для пункта отправления:
req = requests.get("http://autocomplete.travelpayouts.com/places2", params={'term': flight_params['origin']})
orig_data = req.json() 
orig_code = orig_data[0]['code']
from_orig = orig_data[0]['cases']['ro'] # родительный падеж
orig_code, from_orig

('MOW', 'Москвы')

In [6]:
# параметры для пункта назначения
req = requests.get("http://autocomplete.travelpayouts.com/places2", params={'term': flight_params['destination']})
dest_data = req.json() 
dest_code = dest_data[0]['code']
in_dest = dest_data[0]['cases']['vi'] # винительный падеж, куда
dest_code, in_dest

('CPH', 'в Копенгаген')

In [0]:
# основной запрос:
req = requests.get("http://min-prices.aviasales.ru/calendar_preload", 
                   params={'origin': orig_code, 'destination': dest_code, 'one_way': 'true'})

In [0]:
data = req.json()

tickets = data['best_prices']

# сортируем по дате
tickets.sort(key=lambda i: i['depart_date'])

In [9]:
# вывод результата (первые 20 строк)
print(f'Билеты из {from_orig} {in_dest}: ')
print('-'*28)
print(f'|{"Дата вылета":^15}|{"Цена":^10}|')
print('-'*28)
for ticket in tickets[:20:]:
    print(f'|{ticket["depart_date"]:^15}|{ticket["value"]:>10}|')
print('-'*28)

Билеты из Москвы в Копенгаген: 
----------------------------
|  Дата вылета  |   Цена   |
----------------------------
|  2019-07-24   |   11548.0|
|  2019-07-25   |   12219.0|
|  2019-07-26   |   12027.0|
|  2019-07-27   |   11524.0|
|  2019-07-28   |   10215.0|
|  2019-07-29   |   12219.0|
|  2019-07-30   |    8824.0|
|  2019-07-31   |   12091.0|
|  2019-08-01   |    9933.0|
|  2019-08-02   |   12384.0|
|  2019-08-03   |    9933.0|
|  2019-08-04   |   12219.0|
|  2019-08-05   |   12091.0|
|  2019-08-06   |   10601.0|
|  2019-08-07   |    8938.0|
|  2019-08-08   |    9901.0|
|  2019-08-09   |    9909.0|
|  2019-08-10   |   11717.0|
|  2019-08-11   |    8956.0|
|  2019-08-12   |    8181.0|
----------------------------


**Task 2**

В приложении парсинга википедии получить первую ссылку на другую страницу и вывести все значимые слова из неё. Результат записать в файл в форматированном виде


**Task 3  ***

*Научить приложение определять количество ссылок в статье. Спарсить каждую ссылку и результаты записать в отдельные файлы.

In [0]:
import re
import collections
import yaml

In [0]:
def return_wiki_html(topic):
    wiki_request = requests.get(f'https://ru.wikipedia.org/wiki/{topic.capitalize()}')
    return wiki_request.text

In [0]:
def return_words(href):
    req = requests.get(href)
    # print(req.text)
    words = re.findall('[а-яА-Я]{4,}', req.text)
    words_counter = collections.Counter()
    for word in words:
        words_counter[word] += 1
    # for word in words_counter.most_common(10):
    #     print(f'Слово {word[0]} встречается {word[1]} раз')
    return words_counter.most_common(10)

In [0]:
sss = return_wiki_html('игра')

In [14]:
# поиск ссылок, добавил опережающее условие, чтобы исключить автоссылки на wiki
hrefs = re.findall(r'(http[s]?:\/\/(?!.*wiki)[\da-z]+\.[\/\w \.\-\?\=]*)', sss) 
hrefs

['http://antmakarenko.narod.ru/liter/vospm/vosp_det.htm',
 'https://www.webcitation.org/6GZFqFiMD?url=http',
 'http://psychlib.ru/mgppu/zim/ZIM-001-.HTM',
 'http://pedlib.ru/Books/2/0466/2_0466-85.shtml',
 'https://www.webcitation.org/6GZFqpCtA?url=http',
 'http://www.rusnauka.com/13_EISN_2013/Pedagogica/3_136470.doc.htm',
 'https://www.webcitation.org/6GZFrs7lK?url=http',
 'http://modernproblems.org.ru/philosofy/159-games1.html?start=7',
 'https://www.webcitation.org/6GZFsQEVL?url=http',
 'http://modernproblems.org.ru/philosofy/159-games1.html?start=34',
 'https://www.webcitation.org/6GZFtFnpw?url=http',
 'http://rrcdetstvo.ru/content/view/118/119/',
 'http://rrcdetstvo.ru/content/view/118/119/',
 'http://dob.1september.ru/articlef.php?ID=200500510',
 'https://www.webcitation.org/6GZFu0qtr?url=http',
 'http://sbiblio.com/biblio/archive/usnadse_antolo/04.aspx',
 'https://www.webcitation.org/6GZFuzo5J?url=http',
 'http://jorigami.ru/NewDoc/Makarenko_about_games.htm',
 'http://id.ndl.go.

In [15]:
print(f'Количество внешних ссылок со страницы - {len(hrefs)}')

Количество внешних ссылок со страницы - 19


In [0]:
# создаем необходимые файлы в data_path
data_path = 'drive/My Drive/data/'

i=1

for href in hrefs:
  
    with open(data_path + f'href{i}_blocked.yml', 'w') as file:
      
        yaml.dump([href, [list(item) for item in return_words(href)]], file, 
                  Dumper=yaml.Dumper, default_flow_style=False, allow_unicode = True)
        
    i += 1
  